In [1]:
import csv
import numpy as np
import os
import pandas as pd
import time

In [2]:
# SIDER랑 OFFSIDE에 등록된 compound의 DrugBank id 정보
df_sider_id = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv")
df_offside_id = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_3_OFFSIDE_STITCHid_DrugBankid.tsv")
# SIDER랑 OFFSIDE에 등록된 compound의 side effects 정보 (umls id로 표현되어 있음.)
df_sider_umls = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/3_4_SIDER_db_side_effect.tsv")
df_offside_umls = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/4_4_OFFSIDE_db_side_effect.tsv")

In [3]:
print len(df_sider_id)
df_sider_id.head(1)

801


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID
0,CID100002656,CID005479530,DB01212


In [4]:
print len(df_sider_umls)
df_sider_umls.head(1)

72073


,STITCH_compound_id_flat,STITCH_compound_id_stereo,UMLS_concept_id_for_MedDRA,side_effect_name
0,CID100002656,CID005479530,C0000737,Abdominal pain


In [5]:
df_sider = pd.merge(left = df_sider_id, right = df_sider_umls)
df_sider.head(1)

,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID,UMLS_concept_id_for_MedDRA,side_effect_name
0,CID100002656,CID005479530,DB01212,C0000737,Abdominal pain


In [6]:
print len(df_sider)
print len(df_sider.drop_duplicates())
print len(df_sider.drop_duplicates(subset = 'DrugBank_ID'))
print len(df_sider.drop_duplicates(subset = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'DrugBank_ID']))
print len(df_sider.drop_duplicates(subset = ['STITCH_compound_id_flat', 'STITCH_compound_id_stereo', 'UMLS_concept_id_for_MedDRA', 'side_effect_name']))

73951
73951
801
801
72073


In [7]:
print len(df_offside_id)
print len(df_offside_id.drop_duplicates(subset = 'DrugBank_ID'))
df_offside_id.head(1)

558
558


,STITCH_compound_id_stereo,DrugBank_ID
0,CID000000119,DB02530


In [8]:
print len(df_offside_umls)
df_offside_umls.head(1)

193664


,stitch_id,drug,umls_id,event
0,CID000000119,GABA,C0000833,abscess


In [9]:
df_offside = pd.merge(left = df_offside_id, right = df_offside_umls, left_on = 'STITCH_compound_id_stereo', right_on = 'stitch_id')
df_offside.head(1)

,STITCH_compound_id_stereo,DrugBank_ID,stitch_id,drug,umls_id,event
0,CID000000119,DB02530,CID000000119,GABA,C0000833,abscess


In [10]:
print len(df_offside)
print len(df_offside.drop_duplicates(subset = 'DrugBank_ID'))
print len(df_offside.drop_duplicates(subset = ['STITCH_compound_id_stereo', 'DrugBank_ID']))
print len(df_offside.drop_duplicates(subset = ['stitch_id', 'drug', 'umls_id', 'event']))

195460
558
558
193664


In [11]:
all_umls = pd.concat([df_sider_umls['UMLS_concept_id_for_MedDRA'], df_offside_umls['umls_id']])
unique_umls = all_umls.drop_duplicates()

In [12]:
print len(df_sider_umls['UMLS_concept_id_for_MedDRA'].drop_duplicates())
print len(df_offside_umls['umls_id'].drop_duplicates())
print len(all_umls)
print len(unique_umls)

3240
9057
265737
10009


In [13]:
umls_inter = pd.Series(list(set(df_sider_umls['UMLS_concept_id_for_MedDRA']).intersection(set(df_offside_umls['umls_id']))))
len(umls_inter)

2288

In [14]:
# SIDER에서 공통된 feature를 갖고 있는 compound
dup_free_sider = df_sider[df_sider['UMLS_concept_id_for_MedDRA'].isin(umls_inter)].drop_duplicates(subset = 'DrugBank_ID')
print len(dup_free_sider)
dup_free_sider.head(1)

798


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID,UMLS_concept_id_for_MedDRA,side_effect_name
0,CID100002656,CID005479530,DB01212,C0000737,Abdominal pain


In [15]:
# OFFSIDE에서 공통된 feature를 갖고 있는 compound
dup_free_off = df_offside[df_offside['umls_id'].isin(umls_inter)].drop_duplicates(subset = 'DrugBank_ID')
print len(dup_free_off)
dup_free_off.head(1)

558


,STITCH_compound_id_stereo,DrugBank_ID,stitch_id,drug,umls_id,event
0,CID000000119,DB02530,CID000000119,GABA,C0000833,abscess


In [16]:
#SIDER와 OFFSIDE의 공통 feature를 가지는 drug들
drug_with_common_feature = pd.concat([dup_free_sider['DrugBank_ID'], dup_free_off['DrugBank_ID']]).drop_duplicates()

In [17]:
print len(drug_with_common_feature)
drug_with_common_feature.head()

1031


0      DB01212
105    DB00438
184    DB00215
571    DB01327
654    DB00537
Name: DrugBank_ID, dtype: object

In [70]:
umls_inter.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/5_common_feature.tsv", sep='\t', index=False)
umls_inter.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/5_common_feature.tsv", sep='\t', index=False)
drug_with_common_feature.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/6_drugs_with_common_feature.tsv", sep='\t', index=False)
drug_with_common_feature.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/6_drugs_with_common_feature.tsv", sep='\t', index=False)

In [ ]:
# gold drug & features

In [18]:
gold = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/7_gold_association.tsv")

In [29]:
gold_drug_id = gold['drug_id'].drop_duplicates()
print len(gold_drug_id)

471


In [31]:
# SIDER에서 공통된 feature를 갖고 있는 drug들 (feautre정보 포함)
sider_feature = df_sider[df_sider['UMLS_concept_id_for_MedDRA'].isin(umls_inter)]
print len(sider_feature)
print len(sider_feature.drop_duplicates(subset = "DrugBank_ID"))

# 이 중, gold standard drug에 해당하는 것.
sider_gold = sider_feature[sider_feature['DrugBank_ID'].isin(gold_drug_id)][['DrugBank_ID', 'UMLS_concept_id_for_MedDRA', 'side_effect_name']]
print len(sider_gold)
sider_gold.head(1)
print len(sider_gold.drop_duplicates(subset = 'DrugBank_ID'))

58767
798
36477
380


In [34]:
# OFFSIDE에서 공통된 feature를 갖고 있는 drug들 (feature 정보 포함)
offside_feature = df_offside[df_offside['umls_id'].isin(umls_inter)]
print len(offside_feature)
print len(offside_feature.drop_duplicates(subset = 'DrugBank_ID'))

# 이 중, gold standard drug에 해당하는 것.
offside_gold = offside_feature[offside_feature['DrugBank_ID'].isin(gold_drug_id)][['DrugBank_ID', 'umls_id', 'event']]
print len(offside_gold)
offside_gold.head(1)
print len(offside_gold.drop_duplicates(subset = "DrugBank_ID"))

112533
558
69708
287


In [22]:
sider_gold = sider_gold.rename(columns = {'UMLS_concept_id_for_MedDRA' : 'umls_id', 'side_effect_name' : 'event'})
sider_gold.head(1)

,DrugBank_ID,umls_id,event
184,DB00215,C0000737,Abdominal pain


In [23]:
# sider의 offside의 gold drug, side effects의 umls_id, 이름.
frames = [sider_gold, offside_gold]
combined = pd.concat(frames)

In [37]:
print len(combined)
print len(combined.drop_duplicates())
print len(combined.drop_duplicates(subset = ['DrugBank_ID', 'umls_id']))
print len(combined.drop_duplicates(subset = "DrugBank_ID"))
print len(combined.drop_duplicates(subset= "umls_id"))
combined[combined.duplicated(subset = ['DrugBank_ID', 'umls_id'],keep = False)].sort_values(by = ['DrugBank_ID','umls_id']).head()

106185
105676
99083
471
2276


,DrugBank_ID,umls_id,event
57757,DB00165,C0027497,Nausea
9279,DB00165,C0027497,nausea
57758,DB00165,C0030554,Paraesthesia
9305,DB00165,C0030554,formication
57759,DB00165,C0031117,Neuropathy peripheral


In [59]:
combined_name = combined.drop_duplicates(subset = ['DrugBank_ID', 'umls_id'])
print len(combined_name)
combined_name.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/7_gold_drug_feature_name.tsv", sep="\t", index=False)
combined_name.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/7_gold_drug_feature_name.tsv", sep="\t", index=False)

99083


In [25]:
frames2 = [sider_gold[['DrugBank_ID', 'umls_id']], offside_gold[['DrugBank_ID', 'umls_id']]]
combined2 = pd.concat(frames2)

In [61]:
print len(combined2)
combined_id = combined2.drop_duplicates()
print len(combined_id)
combined_id.to_csv("/home/share/aschoi/data/Drug_Repositioning/5_new_drug/7_2_gold_drug_feature_id.tsv", sep="\t", index=False)
combined_id.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/7_2_gold_drug_feature_id.tsv", sep="\t", index=False)


106185
99083


In [ ]:
#여기서 association에 포함되는 drug의 수는?

In [55]:
befor_gold = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/5_before_gold_drug.tsv")
asso = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/4_bi_random_walk_Cdataset_association.tsv")

In [56]:
print len(asso)
asso.head(1)

2532


,disease_id,drug_id
0,D102100,DB00555


In [58]:
filtered_asso = asso[asso['drug_id'].isin(drug_with_common_feature)]

In [59]:
print len(filtered_asso)
filtered_asso.head()

1947


,disease_id,drug_id
0,D102100,DB00555
1,D102100,DB00949
2,D102100,DB01068
3,D102300,DB00190
4,D102300,DB00193


In [60]:
print len(filtered_asso.drop_duplicates(subset='disease_id'))
print len(filtered_asso.drop_duplicates(subset='drug_id'))

364
500


In [35]:
print len(dup_free_sider[dup_free_sider['DrugBank_ID'].isin(befor_gold['drug_id'])])
dup_free_sider[dup_free_sider['DrugBank_ID'].isin(befor_gold['drug_id'])].head(1)

404


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID,UMLS_concept_id_for_MedDRA,side_effect_name
184,CID100002771,CID000002771,DB00215,C0000737,Abdominal pain


In [37]:
# 질병과의 association이 존재하rh, drug feature를 가진 drug가 sider
print len(dup_free_off[dup_free_off['DrugBank_ID'].isin(befor_gold['drug_id'])])
dup_free_off[dup_free_off['DrugBank_ID'].isin(befor_gold['drug_id'])].head(1)

303


,STITCH_compound_id_stereo,DrugBank_ID,stitch_id,drug,umls_id,event
488,CID000000338,DB00936,CID000000338,salicylate,C0001122,Acidosis


In [42]:
si = dup_free_sider[dup_free_sider['DrugBank_ID'].isin(befor_gold['drug_id'])]['DrugBank_ID']

In [45]:
off = dup_free_off[dup_free_off['DrugBank_ID'].isin(befor_gold['drug_id'])]['DrugBank_ID']

In [46]:
len(pd.Series(list(set(si).intersection(set(off)))))

207